In [1]:
import numpy as np
import tensorflow as tf

引入TensorFlow库以后，定义参数，用tf.Variable来定义参数，接下来定义cost代价函数，再初始化一个训练器train，定义train为学习算法，它用梯度下降法优化器使损失函数最小化，使用0.01的学习率，目标是最小化cost损失函数，后面的几行是惯用表达。下一个cell内的代码实际上还没有运行学习算法。

In [12]:
w = tf.Variable(0,dtype=tf.float32)           #定义参数
cost = tf.add(tf.add(w**2,tf.multiply(-10.,w)),25)    #定义损失函数
#cost = w**2-10*w+25
#TensorFlow还重载了一般的加减运算，所以可以直接写表达式。一旦w被定义为TensorFlow变量，加减等各种运算都重载了
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)   #定义train为学习算法

init = tf.global_variables_initializer()
session = tf.Session()   #这样就开启了一个TensorFlow session
session.run(init)        #初始化全局变量
print(session.run(w))       #让TensorFlow评估一个变量

0.0


运行上面代码，让工具对w参数进行评估为0，因为我们什么都还没有运行。

In [13]:
session.run(train)           #这一步所做的就是运行一步梯度下降法
print(session.run(w))       #运行了一步梯度下降之后，再评估w的值

0.099999994


In [14]:
#接下来运行梯度下降1000次迭代
for i in range(1000):
    session.run(train) 
print(session.run(w)) 

4.9999886


以上对TensorFlow程序的大致结构应该有了解了，需要注意的地方时，w是我们想要优化的参数，因此把它称为变量，需要做的就是定义一个损失函数，使用add和multiply之类的函数，TensorFlow知道如何对add以及multiply等函数进行求导，这就是你为什么只需要实现前向传播，TensorFlow能自己弄明白如何做反向传播和梯度计算，因为它已经内置在add之类的函数里了。

Tensorflow还有一个特点就是，这个例子是最小化w的固定函数，其中一个需要最小化的函数就是关于训练集的函数，当给你一些训练数据x，当你训练一个神经网络时 训练数据x会变化，怎样才能把训练数据导入一个TensorFlow的程序？首先定义一个x，扮演训练数据的角色。事实上训练数据有x有y，这里只有x，作为那个二次损失函数的系数变化训练集,所以定义为(3,1)维数组。将x系统放到cost函数中。placeholder的作用就是告诉TensorFlow稍后会为x提供数值，再定义一个数组，coefficient用来给x赋值。接下来就需要某种方式把这个系数数组接入变量x，在启动训练这一步实现。

In [15]:
coefficient = np.array([[1.],[-10.],[25.]])

w = tf.Variable(0,dtype=tf.float32)           #定义参数
x = tf.placeholder(tf.float32,[3,1])
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)   #定义train为学习算法

init = tf.global_variables_initializer()
session = tf.Session()   #这样就开启了一个TensorFlow session
session.run(init)        #初始化全局变量
print(session.run(w))       #让TensorFlow评估一个变量

0.0


In [18]:
session.run(train,feed_dict={x:coefficient})           
print(session.run(w))    

0.099999994


In [19]:
for i in range(1000):
    session.run(train,feed_dict={x:coefficient}) 
print(session.run(w)) 

4.9999886


根据提供可变化的参数，模拟训练集，输入相同的数值系数时结果没有改变，也可以在coefficient处修改数值，尝试其他二次函数的最小值。

后面会发现TensorFlow中的placeholder是你之后会赋值的变量，这是把训练数据导入代价函数的一个很方便的办法，上面这个例子就是把数据导入代价函数的语法，当你迭代训练的时候，在feed_dict中把x的值设置成coefficients，当你在做最小批梯度下降的时候，由于你要在每一次迭代的时候代入一个最小批，所以你需要用feed_dict把训练集的不同的子集，不同的最小批代入需要数据的代价方程，希望这让你们对TensorFlow能做的事情有一个初步了解。


TensorFlow如此强大是因为你只需要指定如何计算代价函数，只需要一两行代码就能求导，使用梯度优化器或者Adam优化器或者其它优化器，在声明train那可以更改。
最后一件需要注意的是，关于init那三行代码在TensorFlow中是比较惯用的，也可以用下面这个版本：
```python
with tf.Session() as session:
    session.run(init)
    print(session.run(w))
```
但是这个with结构也用在TensorFlow的其它一些程序中,它和上面的init的代码是差不多的意思，不过当在执行这个内循环有异常出现的时候，Python中的这个with命令，在清理的时候会好用一些。

关于这行代码：
```python
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]
```
到底在做什么呢，TensorFlow程序的核心是计算代价函数，之后TensorFlow会自动求导，并计算出如何最小化代价函数，所以这个等式 或者说这段代码实际上做的是让TensorFlow构建一个计算图，计算图会执行以下操作，先取出x[0][0]，再将w进行平方，再两者相乘等等类似于一个说明单步运算的流程图，最后得到代价函数。

TensorFlow的好处是正如像上面的图一样用前向传播来计算代价函数，TensorFlow已经内置了，所有必须的后向传播方程，所以请记住训练神经网络有一套前向传播方程
和后向传播方程，像TensorFlow这样的编程框架已经内置必须的后向传播方程，这就是为什么当你使用内置的函数来计算前向传播方程的时候，即使是非常复杂的函数，它也能自动使用后向传播函数来执行后向传播并求导，这就是为什么你不需要显示地执行后向传播，这是编程框架有助于提高你的效率。